In [1]:
from tespy.networks import Network
from tespy.connections import Connection, Ref, Bus
from tespy.components import Source, Sink, CycleCloser, Compressor, Valve
from tespy.components import MovingBoundaryHeatExchanger as HeatExchanger


nw = Network(T_unit="C", p_unit="bar", h_unit="kJ / kg", m_unit="kg / s")

air_in = Source("air inlet")
air_out = Sink("air outlet")

water_in = Source("water inlet")
water_out = Sink("water outlet")

air_hx = HeatExchanger("AIR_HX")
comp1 = Compressor("COMP1")
valve1 = Valve("VAL1")
cc1 = CycleCloser("cc")

ihx = HeatExchanger("IHX")

steam_gen = HeatExchanger("STEAM_GEN")
comp2 = Compressor("COMP2")
valve2 = Valve("VAL2")
cc2 = CycleCloser("cc2")

c11 = Connection(air_in, "out1", air_hx, "in1", label="11")
c12 = Connection(air_hx, "out1", air_out, "in1", label="12")

c21 = Connection(air_hx, "out2", comp1, "in1", label="21")
c22 = Connection(comp1, "out1", ihx, "in1", label="22")
c22c = Connection(ihx, "out1", cc1, "in1", label="22c")
c23 = Connection(cc1, "out1", valve1, "in1", label="23")
c24 = Connection(valve1, "out1", air_hx, "in2", label="24")

c31 = Connection(ihx, "out2", comp2, "in1", label="31")
c32 = Connection(comp2, "out1", steam_gen, "in1", label="32")
c32c = Connection(steam_gen, "out1", cc2, "in1", label="32c")
c33 = Connection(cc2, "out1", valve2, "in1", label="33")
c34 = Connection(valve2, "out1", ihx, "in2", label="34")

c41 = Connection(water_in, "out1", steam_gen, "in2", label="41")
c42 = Connection(steam_gen, "out2", water_out, "in1", label="42")

nw.add_conns(c21, c22, c22c, c23, c24)
nw.add_conns(c11, c12)
nw.add_conns(c31, c32, c32c, c33, c34)
nw.add_conns(c41, c42)

# Simulation with starting values

c11.set_attr(fluid={"Ar": 0.0129, "CO2": 0.0005, "N2": 0.7552, "O2": 0.2314}, T=20, p=1.013)
c12.set_attr(T=Ref(c11, 1, -5))

c21.set_attr(fluid={"R245FA": 1}, h=417)
c22.set_attr(p=6.4)
c23.set_attr(h=290)
c24.set_attr(p=0.823)

c31.set_attr(fluid={"R1233zdE": 1}, h=451)
c32.set_attr(p=17.5)
c33.set_attr(h=364)
c34.set_attr(p=4.1)

c41.set_attr(fluid={"water": 1}, p=2, x=0, m=1)
c42.set_attr(h=2706)

comp1.set_attr(eta_s=0.8)
comp2.set_attr(eta_s=0.8)

steam_gen.set_attr(pr1=0.95, pr2=1)
air_hx.set_attr(pr1=1, pr2=0.95)
ihx.set_attr(pr1=0.95, pr2=0.95)

# nw.solve("design")

# Simulation with fixed values

c21.set_attr(h=None, Td_bp=5)
c23.set_attr(h=None, Td_bp=-5)
c24.set_attr(p=None)

c31.set_attr(h=None, Td_bp=5)
c32.set_attr(p=None)
c33.set_attr(h=None, x=0)
c34.set_attr(p=None, T=60)

c42.set_attr(h=None, x=1)

air_hx.set_attr(ttd_l=5)
steam_gen.set_attr(ttd_l=5)

nw.solve("design")

c22.set_attr(p=None)
ihx.set_attr(ttd_l=5)

power_input = Bus("power input")
power_input.add_comps(
    {"comp": comp1, "base": "bus", "char": 0.985},
    {"comp": comp2, "base": "bus", "char": 0.985}
)

nw.add_busses(power_input)

nw.solve("design")
nw.print_results()

Q_out = c42.m.val * (c42.h.val - c41.h.val)
COP2 = c42.m.val * (c42.h.val - c41.h.val) / (comp2.P.val*1e-3)
COP1 = c31.m.val * (c31.h.val - c34.h.val) / (comp1.P.val*1e-3)
COP = c42.m.val * (c42.h.val - c41.h.val) / (power_input.P.val*1e-3)

print("Q = ", round(Q_out, 1), "kW")
print("COP = ", round(COP, 3))
print("COP1 = ", round(COP1, 3))
print("COP2 = ", round(COP2, 3))

# assert convergence of calculation
nw.assert_convergence()

# ambient conditions
p0 = c11.p.val * 1e5
T0 = c11.T.val + 273.15

# economic parameters
# Define the CEPCI values for cost correction.
CEPCI_2013 = 567.3
CEPCI_2023 = 797.9
CEPCI_factor = CEPCI_2023 / CEPCI_2013

# Define default values for electricity price and full load hours.
default_elec_price = 40.0   # cent/kWh
default_tau = 5500          # hours/year

# Define economic parameters.
r_n = 0.02                  # Cost elevation rate
i_eff = 0.08                # Interest rate
n = 20                      # Number of years
omc_relative = 0.03         # Relative operation and maintenance costs (compared to PEC)

fuel = {
    "inputs": [
        'power input__motor_of_COMP1',
        'power input__motor_of_COMP2'
    ],
    "outputs": []
}

product = {
    "inputs": ['42'],
    "outputs": ['41']
}

loss = {
    "inputs": ['12'],
    "outputs": ['11']
}

####################################################


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.94e+06   | 0 %        | 7.31e+01   | 7.24e+08   | 7.45e+06   | 0.00e+00   | 0.00e+00   
 2     | 5.12e+06   | 0 %        | 2.87e+01   | 1.40e+07   | 1.07e+06   | 0.00e+00   | 0.00e+00   
 3     | 2.27e+06   | 0 %        | 2.43e+03   | 9.84e+07   | 1.44e+06   | 0.00e+00   | 0.00e+00   
 4     | 1.79e+07   | 0 %        | 2.91e+03   | 6.14e+05   | 8.61e+04   | 0.00e+00   | 0.00e+00   
 5     | 6.72e+05   | 1 %        | 1.56e+02   | 3.97e+04   | 6.32e+02   | 0.00e+00   | 0.00e+00   
 6     | 2.29e+03   | 29 %       | 5.82e-01   | 3.43e+03   | 9.74e+01   | 0.00e+00   | 0.00e+00   
 7     | 1.74e+00   | 63 %       | 3.08e-02   | 1.69e+02   | 1.20e+01   | 0.00e+00   | 0.00e+00   
 8     | 2.50e-03   | 95 %       | 1.48e-03   | 8.93e+00   | 5.80e-01   | 0.00e+00   | 0.00e+00   
 9     | 

The MovingBoundaryHeatExchanger instance AIR_HX is defined for constant pressure. The identification of the heat transfer sections might be wrong in case phase changes are involved in the heat transfer process.
The MovingBoundaryHeatExchanger instance IHX is defined for constant pressure. The identification of the heat transfer sections might be wrong in case phase changes are involved in the heat transfer process.
The MovingBoundaryHeatExchanger instance IHX is defined for constant pressure. The identification of the heat transfer sections might be wrong in case phase changes are involved in the heat transfer process.
The MovingBoundaryHeatExchanger instance STEAM_GEN is defined for constant pressure. The identification of the heat transfer sections might be wrong in case phase changes are involved in the heat transfer process.
The MovingBoundaryHeatExchanger instance AIR_HX is defined for constant pressure. The identification of the heat transfer sections might be wrong in case phase


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 8.20e-02   | 78 %       | 1.16e-01   | 1.40e+03   | 1.27e+02   | 0.00e+00   | 0.00e+00   
 2     | 4.69e-01   | 70 %       | 2.98e-05   | 1.16e+00   | 2.00e-02   | 0.00e+00   | 0.00e+00   
 3     | 9.74e-08   | 100 %      | 2.60e-11   | 6.50e-07   | 1.67e-08   | 0.00e+00   | 0.00e+00   
 4     | 2.55e-08   | 100 %      | 3.55e-12   | 5.95e-08   | 3.04e-09   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.10 s, Iterations per second: 41.83

##### RESULTS (MovingBoundaryHeatExchanger) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+-------------+------------------+----------------+----------------+---------------------+-----

In [2]:
from exerpy import ExergyAnalysis

ean = ExergyAnalysis.from_tespy(nw, Tamb=T0, pamb=p0)
ean.analyse(fuel, product, loss)

Component class Source not available in exerpy.
Component class Sink not available in exerpy.


In [3]:
import pandas as pd

In [4]:
cols = ["chemical", "physical", "massless"]
group_data = {
    "E_F": pd.DataFrame(columns=[cols])
}
for label, data in ean._connection_data.items():
    if data["source_component"] not in group_data:
        group_data[data["source_component"]] = pd.DataFrame(columns=cols)

    if "E_PH" not in data and "E_CH" not in data:
        group_data[data["source_component"]].loc[data["target_component"]] = [
            0, 0, data.get("E", 0)
        ]
    else:
        group_data[data["source_component"]].loc[data["target_component"]] = [
            data.get("E_CH", 0), data.get("E_PH", 0), 0
        ]

for connlabel in ean.E_F_dict["inputs"]:
    group_data["E_F"].loc[ean._connection_data[connlabel]["target_component"]] = [0, 0, 0]

for component_name, component in ean.components.items():
    if not hasattr(component, "E_D"):
        E_D = 0
    else:
        E_D = component.E_D
    group_data[component_name].loc["E_D"] = [0, 0, E_D]

In [ ]:
def generate_plotly_sankey_input(
        group_data, node_order=[], colors={}, display_thresold=1e-3,
        disaggregate_flows=False):



    # for fkt_group, data in group_data.copy().items():
    #     mask = data.abs().sum(axis=1) >= display_thresold
    #     group_data[fkt_group] = group_data[fkt_group].loc[mask]

    # self.remove_transit_groups(group_data)

    if len(node_order) == 0:
        node_order = (
            ['E_F']
            + list(set([k for data in group_data.values() for k in data.index if k not in ["E_F", "E_P", "E_L", "E_D"]] + [k for k in group_data if k not in ["E_F", "E_P", "E_L", "E_D"]]))
            + ['E_P', 'E_L', 'E_D']
        )
    else:
        missing = []
        for node in group_data:
            if node not in node_order:
                missing += [node]

        # if len(missing) > 0:
        #     msg = (
        #         'The list of nodes passed is missing the following '
        #         'nodes: "' + '", "'.join(missing) + '".')
        #     logger.error(msg)
        #     raise ValueError(msg)

    # colordict = {
    #     "E_F": "rgba(242, 142, 43, 0.90)",
    #     "E_P": "rgba(118, 183, 178, 0.90)",
    #     "E_D": "rgba(176, 122, 161, 0.90)",
    #     "E_L": "rgba(156, 117, 95, 0.90)",
    #     "combustion-gas": "rgba(237, 201, 72, 0.90)",
    #     "non-combustion-gas": "rgba(186, 176, 172, 0.90)",
    #     "two-phase-fluid": "rgba(89, 161, 79, 0.90)",
    #     "incompressible": "rgba(255, 157, 167, 0.90)",
    #     "work": "rgba(78, 121, 167, 0.90)",
    #     "heat": "rgba(225, 87, 89, 0.90)",
    #     np.nan: "rgba(100, 100, 100, 1.00)"
    # }
    # colordict.update(colors)

    links = {
        'source': [],
        'target': [],
        'value': [],
        'color': []
    }
    for fkt_group, data in group_data.items():
        source_id = node_order.index(fkt_group)
        for target in data.index:
            for col in data.columns:
                # how to aggregate here?
                if data.loc[target, col] > 0.:
                    links['source'] += [source_id]
                    links['target'] += [node_order.index(target)]
                    links['value'] += [data.loc[target, col]]
                    # links['color'].append(colordict[target[1]])

    return links, node_order

In [6]:
links, nodes = generate_plotly_sankey_input(group_data)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(go.Sankey(
    arrangement="snap",
    node={
        "label": nodes,
        'pad': 11,
        'color': 'orange'},
    link=links))
fig